In [ ]:
import os
import scipp as sc
import scippneutron as scn
from scippneutron.tof import conversions
from ess import amor, reflectometry as refl

In [ ]:
local_data_path = os.path.join('ess-notebooks', 'amor')
data_dir = os.path.join('data', local_data_path)
data_file = os.path.join(data_dir, 'sample.nxs')

sample = amor.io.load(data_file)
sample

In [ ]:
sc.plot(sample)

In [ ]:
graph = refl.conversions.reflectometry_graph()
graph

In [ ]:
sc.show_graph(graph, simplified=True)

In [ ]:
sample.coords["gravity"] = sc.vector(value=[0, -1, 0]) * sc.constants.g
sample

In [ ]:
q_with_gravity = sample.transform_coords(["Q"], graph=graph)
q_with_gravity

In [ ]:
q_edges = sc.linspace(dim='Q', start=0.008, stop=0.08,
                      num=201, unit='1/angstrom')
q_binned_with_g = sc.bin(q_with_gravity, edges=[q_edges])
q_summed_with_g = q_binned_with_g.sum('detector_id')
sc.plot(q_summed_with_g["wavelength", 0], norm="log")